In [ ]:
# for debug only
import gzip

def read_gzipped_file(file_path, num_lines=5):
    lines = []
    with gzip.open(file_path, "rt") as gzfile:
        for _ in range(num_lines):
            line = gzfile.readline().strip()
            if not line:  # Break the loop if we reach the end of the file
                break
            lines.append(line)
    return lines

# gz_file_path = os.path.join(cur_work_dir,"GSM4008632_Adult-Lung3-3_dge.txt.gz")

# num_lines_to_read = 5
# lines = read_gzipped_file(gz_file_path, num_lines=num_lines_to_read)
# print(lines)


In [ ]:
import pandas as pd
import gzip

def read_gzipped_dge(gz_file_path):
    # Initialize lists to store the data
    gene_names = []
    cell_names = []
    umi_counts = []

    # Read the gzipped DGE data
    with gzip.open(gz_file_path, "rt") as gzfile:
        # Skip the header line with column names
        next(gzfile)

        for line in gzfile:
# debug
#         for _ in range(5):
#             line = gzfile.readline().strip()
            fields = line.strip().split("\t")
            gene_names.append(fields[0])
            cell_names.append(fields[1])
            umi_counts.append([int(count) for count in fields[2:]])

    # Create the pandas DataFrame
    df = pd.DataFrame(umi_counts, index=gene_names, columns=cell_names)

    return df

# # Example usage:
# gz_dge_file_path = "path/to/your/gzipped_dge_data.txt.gz"
# exp_mtx = read_gzipped_dge(gz_dge_file_path)


In [5]:
# gene_names_to_ensembl_ids(["A1BG-AS1"])

record is  {'Count': '3', 'RetMax': '3', 'RetStart': '0', 'IdList': ['10211', '574436', '503538'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'A1BG-AS1[All Fields]', 'Field': 'All Fields', 'Count': '3', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'A1BG-AS1[All Fields]'}
ensembl_ids is  ['10211']


['10211']

In [ ]:
import numpy as np
import gzip
import loompy

def create_loom_from_gz_dge_mtx(gz_file_path, loom_file_path,labels):
    # Read and process the gzipped DGE data
    # debug
    debug_num_of_cells = 2 #genes and cell cut off
    label_idx = 0
    # read the file to mtx
    gene_names = []
    cell_names = []
    umi_counts = []
    labels_list = []
    print("labels is {}".format(labels))

    # when adding more file, I need to increase the label
    with gzip.open(gz_file_path, "rt") as gzfile:
        # Skip the header line with column names
        cell_names = next(gzfile)
        print("all cell_names are".format(cell_names))
        cell_names = cell_names.split("\t")
        print("cell_names[0:5] after split is {}".format(cell_names[0:5]))
        cell_names = cell_names[1:1+debug_num_of_cells]
        print("cell_names after removing the first one is {}".format(cell_names[0:debug_num_of_cells]))
        print("len(cell_names)) is {}".format(len(cell_names)))

#         for line in gzfile:
 # debug
        for _ in range(debug_num_of_cells): # reviewing debug number of genes
            line = gzfile.readline().strip()
            fields = line.strip().split("\t")
            print("fields[0:1+debug_num_of_cells] are {}".format(fields[0:1+len(cell_names)]))
            gene = fields[0]
            print("gene is {}".format(gene))
            ensemble_id = gene_symbol_to_ensembl_id(fields[0])
            if (ensemble_id == None):
                print("!!!skipping gene {}. couldnt find enseble ID for it".format(gene))
                continue
            print("ensemble_id is {}".format(ensemble_id))
            gene_names.append(ensemble_id) # convert the gene name to ensembl id
            print("gene_names ensemble_id is {}".format(gene_names))
            # cell_names.append(fields[1:1+debug_num_of_cells])
            # Convert the vector values from str to int using list comprehension
            umi_counts_int = [int(value) for value in fields[1:1+debug_num_of_cells]]
            # Calculate the sum using the sum() function
            # umi_counts_sum = sum(umi_counts_int)
            umi_counts.append(umi_counts_int)
                # [int(count) for count in fields[1:1+debug_num_of_cells]])
            print("umi_counts is {}".format(umi_counts))
            labels_list.append(labels[label_idx])
            print("labels_list is {}".format(labels_list))

    # Convert the lists to NumPy arrays for efficient processing
    gene_names_arr = np.array(gene_names)
    print("gene_names_arr np array is {}".format(gene_names_arr))
    cell_names_arr = np.array(cell_names) #[0:debug_num_of_cells])
    print("cell_names_arr np array is {}".format(cell_names_arr))
    umi_counts_arr = np.array(umi_counts) #[0:debug_num_of_cells])
    print("umi_counts_arr np array is {}".format(umi_counts_arr))
    print("umi_counts_arr dim is {}".format(umi_counts_arr.shape))
    tissue_type_arr = np.array(labels_list)
    print("tissue_type_arr np array is {}".format(tissue_type_arr))

    # Create an empty array for the default layer
#     default_layer = np.empty((len(cell_names), len(gene_names)), dtype=int)

    # Create the Loom file and write data
    # layers_arr = {"": umi_counts_arr}
    layers_arr = umi_counts_arr
    row_attrs = {"ensembl_id": gene_names_arr}
    col_attrs = {
      'cell_id': cell_names_arr,
      'n_counts': umi_counts_arr,
      'tissue_type': tissue_type_arr
    }
#     {"CellID": cell_names}

    print("umi_counts_arr shape is {}".format(umi_counts_arr.shape))
    print("cell_names_arr shape is {}".format(cell_names_arr.shape))
    print("umi_counts_arr shape is {}".format(umi_counts_arr.shape))
    print("tissue_type_arr shape is {}".format(tissue_type_arr.shape))
    print("loom_file_path is {}".format(loom_file_path))
    print("col_attrs is {}".format(col_attrs))
    shape = layers_arr.shape
    print("layers_arr shape is {}".format(shape))
    print("layers_arr shape[0] is {}".format(shape[0]))
    print("layers_arr shape[1] is {}".format(shape[1]))
    
    loompy.create(loom_file_path, layers=layers_arr, row_attrs=row_attrs, col_attrs=col_attrs)

# # usage:
# gz_dge_file_path = os.path.join(cur_work_dir,"GSM4008632_Adult-Lung3-3_dge.txt.gz")
# loom_file_path = os.path.join(cur_work_dir,"GSM4008632_Adult-Lung3-3_dge.txt.gz".split(".")[0]) #"scRNA-seq.loom"

# create_loom_from_gz_dge(gz_dge_file_path, loom_file_path)


In [ ]:
import numpy as np
import loompy

def load_loom_file(loom_file_path):
    # Connect to the Loom file
    with loompy.connect(loom_file_path) as ds:
        # Access the gene expression data and other attributes
        gene_expression_matrix = ds[:, :]  # Get the entire gene expression matrix
        gene_names = ds.ra.ensembl_id      # Get gene names from the row attributes
        cell_names = ds.ca.cell_id         # Get cell names from the column attributes
        tissue_names = ds.ca.tissue_type   # Get tissue type names from the column attributes

    return gene_expression_matrix, gene_names, cell_names, tissue_names

# # Example usage:
# # loom_file_path = "path/to/your/scRNA-seq.loom"
# gene_expression_matrix, gene_names, cell_names = load_loom_file(loom_file_path)

# # Now you can work with the loaded data
# # For example, you can print the first few rows of the gene expression matrix:
# print(gene_expression_matrix[:5, :5])
# print(gene_names)

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
['A1BG-AS1' 'A2M' 'A2M-AS1' 'A4GALT' 'A4GNT']


In [ ]:
# "GSM4008632_Adult-Lung3-3_dge.txt.gz".split(".")[0]

In [1]:
import requests

def gene_symbol_to_ensembl_id(gene_symbol):
    # Ensembl REST API endpoint
    ensembl_api_url = "https://rest.ensembl.org"

    # Set the headers for the request
    headers = {
        "Content-Type": "application/json",
    }

    # Perform the gene symbol to Ensembl ID conversion
    ext = f"/lookup/symbol/human/{gene_symbol}?expand=1"
    response = requests.get(ensembl_api_url + ext, headers=headers)

    if response.ok:
        data = response.json()
        if "id" in data:
            return data["id"]
    else:
        response.raise_for_status()
        if gene =="AAED1":
            return "ENSG00000158122"
        pass

    return None

# if __name__ == "__main__":
#     gene_symbol = "BRCA1"
#     ensembl_id = gene_symbol_to_ensembl_id(gene_symbol)

#     if ensembl_id:
#         print(f"Ensembl ID for gene symbol {gene_symbol}: {ensembl_id}")
#     else:
#         print(f"No Ensembl ID found for gene symbol {gene_symbol}.")


In [ ]:
import shutil

def safe_remove_directory(directory_path):
    try:
        shutil.rmtree(directory_path, ignore_errors=True)
        print(f"Directory '{directory_path}' and its contents have been removed.")
    except Exception as e:
        print(f"An error occurred while removing '{directory_path}': {e}")

# if __name__ == "__main__":
#     # Replace 'path_to_directory' with the actual path of the directory you want to remove.
#     directory_path = "path_to_directory"
#     safe_remove_directory(directory_path)